mobilenet_v3_large, SMOTE, ReLu, Adam

In [2]:
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from torchinfo import summary
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import torch.optim as optim
from torchvision.models import swin_s
from torchvision.models import mobilenet_v3_large
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from collections import OrderedDict
from tqdm import tqdm

In [3]:
from elpv_reader import load_dataset
images, proba, types = load_dataset()

In [4]:
# check version number
import imblearn
from imblearn.over_sampling import SMOTE

train_set, test_set, train_lables, test_lables = \
train_test_split(images, proba, test_size=0.25,random_state=42)

my_mapping = {0.0:0, 0.3333333333333333:1, 0.6666666666666666:2, 1.0:3}
train_lables_new = []
for i in range(len(train_set)):
    train_lables_new.append(my_mapping[train_lables[i]])
train_lables_new = np.array(train_lables_new)
print(train_lables_new)

for i in range(len(test_set)):
    test_lables[i] = my_mapping[test_lables[i]]

[0 3 0 ... 0 1 3]


In [5]:
count_list = [0,0,0,0]
for i in range(len(train_set)):
    if train_lables_new[i] == 0:
        count_list[0] += 1
    if train_lables_new[i] == 1:
        count_list[1] += 1
    if train_lables_new[i] == 2:
        count_list[2] += 1
    if train_lables_new[i] == 3:
        count_list[3] += 1 
for i, j in zip(['The 0.0 has:', 'The 0.33 has:', 'The 0.66 has:', 'The 1.0 has:'], count_list):
    print(i, j)

The 0.0 has: 1136
The 0.33 has: 222
The 0.66 has: 75
The 1.0 has: 535


In [6]:
train_set = np.array(train_set)
new_set_2d = train_set.reshape(train_set.shape[0], -1)
# sampling_strategy = {0: 1136, 1: 800, 2: 800, 3: 800}
# oversample = SMOTE(sampling_strategy=sampling_strategy)
oversample = SMOTE(sampling_strategy='not majority')
new_set1, new_labels1 =  oversample.fit_resample(new_set_2d, train_lables_new)

In [7]:
print(new_set1.shape)
new_set1_3d = new_set1.reshape((-1, 300, 300))
print(new_set1_3d.shape)

# for i in range(len(new_labels1)):
#     print(new_labels1[i])

count = 0
for i in range(len(new_set1_3d)):
    # plt.imshow(new_set1_3d[i], cmap='gray')
    # plt.show()
    # print(new_labels1[i])
    count += 1
print(count)

count_list = [0,0,0,0]
for i in range(len(new_set1_3d)):
    if new_labels1[i] == 0:
        count_list[0] += 1
    if new_labels1[i] == 1:
        count_list[1] += 1
    if new_labels1[i] == 2:
        count_list[2] += 1
    if new_labels1[i] == 3:
        count_list[3] += 1 
for i, j in zip(['The 0.0 has:', 'The 0.33 has:', 'The 0.66 has:', 'The 1.0 has:'], count_list):
    print(i, j)

(4544, 90000)
(4544, 300, 300)
4544
The 0.0 has: 1136
The 0.33 has: 1136
The 0.66 has: 1136
The 1.0 has: 1136


In [7]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=False):
        self.images = images
        self.label_mapping = {0: np.eye(4)[0], 1: np.eye(4)[1],\
                              2: np.eye(4)[2], 3: np.eye(4)[3]}
        self.labels = [self.label_mapping[label] for label in labels]
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = Image.fromarray(image)

        # 如果图像为灰度图像，将其复制到三个通道，转换为 RGB 图像
        if image.mode == 'L':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
    ])


custom_train_dataset = CustomDataset(new_set1_3d, new_labels1, transform)
custom_test_dataset = CustomDataset(test_set, test_lables, transform)


batch_size = 32
train_loader = DataLoader(custom_train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(custom_test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda")  
else:
    device = torch.device("cpu")  

model = mobilenet_v3_large(weights=True)

classifier1 = nn.Sequential(OrderedDict([('fc1', nn.Linear(960, 4)),
                           ('output', nn.ReLU())
                          ]))
    
model.classifier = classifier1

model.to(device)
print()

d:\anaconda\envs\torchtest\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:

lossfunc = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

save_path = '.\model.pth'
best_accuracy = 79
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # 移动数据到 CUDA
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lossfunc(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Training Loss: {train_loss:.4f}')


    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            _, real_labels = torch.max(labels, 1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(real_labels.cpu().numpy())

    
    accuracy = accuracy_score(all_labels, all_predictions)
    conf_matrix = confusion_matrix(all_labels, all_predictions)

    if best_accuracy < accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), save_path)
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Saved Best Model (Best Accuracy: {best_accuracy:.4f})')

    print(f'Accuracy: {accuracy:.4f}')
    print('Confusion Matrix:')
    print(conf_matrix)

100%|██████████| 142/142 [00:16<00:00,  8.41it/s]


Epoch 1/20 - Training Loss: 1.3166
Accuracy: 0.6448
Confusion Matrix:
[[350   1   2  19]
 [ 70   0   1   2]
 [ 30   0   0   1]
 [107   0   0  73]]


100%|██████████| 142/142 [00:14<00:00, 10.11it/s]


Epoch 2/20 - Training Loss: 1.1774
Accuracy: 0.6784
Confusion Matrix:
[[353   0   4  15]
 [ 69   0   2   2]
 [ 24   0   1   6]
 [ 87   0   2  91]]


100%|██████████| 142/142 [00:14<00:00, 10.08it/s]


...

Epoch 18/20 - Training Loss: 0.3484
Accuracy: 0.7866
Confusion Matrix:
[[347   9   0  16]
 [ 36  28   5   4]
 [ 15   5   6   5]
 [ 36   6   3 135]]


100%|██████████| 142/142 [00:14<00:00,  9.50it/s]


Epoch 19/20 - Training Loss: 0.3292
Accuracy: 0.7835
Confusion Matrix:
[[354  11   0   7]
 [ 38  27   4   4]
 [ 18   6   4   3]
 [ 39   9   3 129]]


100%|██████████| 142/142 [00:15<00:00,  9.19it/s]


Epoch 20/20 - Training Loss: 0.3114
Accuracy: 0.7912
Confusion Matrix:
[[350  10   1  11]
 [ 35  29   5   4]
 [ 14   5   7   5]
 [ 37   7   3 133]]


In [11]:
# 创建两个子集，分别存储mono和poly的标签和预测
mono_subset_labels = [label for label, img_type in zip(all_labels, types) if img_type == "mono"]
mono_subset_predictions = [pred for pred, img_type in zip(all_predictions, types) if img_type == "mono"]
poly_subset_labels = [label for label, img_type in zip(all_labels, types) if img_type == "poly"]
poly_subset_predictions = [pred for pred, img_type in zip(all_predictions, types) if img_type == "poly"]

# 计算不同类型的准确率和混淆矩阵
accuracy = accuracy_score(all_labels, all_predictions)
accuracy_mono = accuracy_score(mono_subset_labels, mono_subset_predictions)
accuracy_poly = accuracy_score(poly_subset_labels, poly_subset_predictions)
conf_matrix = confusion_matrix(all_labels, all_predictions)
conf_matrix_mono = confusion_matrix(mono_subset_labels, mono_subset_predictions)
conf_matrix_poly = confusion_matrix(poly_subset_labels, poly_subset_predictions)
f1_total = f1_score(all_labels, all_predictions, average='macro')
f1_mono = f1_score(mono_subset_labels, mono_subset_predictions, average='macro')
f1_poly = f1_score(poly_subset_labels, poly_subset_predictions, average='macro')

print('The total accuracy is:', accuracy)
print()
print('The accuracy of mono type is:', accuracy_mono)
print()
print('The accuracy of poly type is:', accuracy_poly)
print()
print('The total confusion matrix is:\n', conf_matrix)
print()
print('The confusion matrix of mono type is:\n', conf_matrix_mono)
print()
print('The confusion matrix of poly type is:\n', conf_matrix_poly)
print()
print('The total F1 Score is:', f1_total)
print()
print('The F1 Score of mono type is:', f1_mono)
print()
print('The F1 Score of poly type is:', f1_poly)


The total accuracy is: 0.7911585365853658

The accuracy of mono type is: 0.7708333333333334

The accuracy of poly type is: 0.8028846153846154

The total confusion matrix is:
 [[350  10   1  11]
 [ 35  29   5   4]
 [ 14   5   7   5]
 [ 37   7   3 133]]

The confusion matrix of mono type is:
 [[127   1   1   5]
 [ 17   9   2   1]
 [  5   2   3   2]
 [ 15   3   1  46]]

The confusion matrix of poly type is:
 [[223   9   0   6]
 [ 18  20   3   3]
 [  9   3   4   3]
 [ 22   4   2  87]]

The total F1 Score is: 0.607687427092892

The F1 Score of mono type is: 0.5875846549402974

The F1 Score of poly type is: 0.6183270504463467
